<a href="https://colab.research.google.com/github/MonMon120/test/blob/main/sexism_MLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Data preparation part**

In [ ]:
!pip3 install --quiet "tensorflow-text==2.14.*" # compatibility issues with higher versions

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.5/6.5 MB 24.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 489.9/489.9 MB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 29.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 440.7/440.7 kB 20.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 37.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 21.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 24.5 MB/s eta 0:00:00


**Import dataset and Bert model**

In [ ]:
import tensorflow as ts
import tensorflow_hub as hub
import tensorflow_text as text

import numpy as np
import pandas as pd

#Bert model
bert_preprocess = hub.KerasLayer("/content/drive/MyDrive/ai models/bert_preprocess")
bert_encoder = hub.KerasLayer("/content/drive/MyDrive/ai models/bert_model")

#Dataset
df = pd.read_csv('/content/drive/MyDrive/datasets/edos_labelled_aggregated.csv')
df.head()

,rewire_id,text,label_sexist,label_category,label_vector,split
0,sexism2022_english-9609,"In Nigeria, if you rape a woman, the men rape ...",not sexist,none,none,dev
1,sexism2022_english-16993,"Then, she's a keeper. 😉",not sexist,none,none,train
2,sexism2022_english-13149,This is like the Metallica video where the poo...,not sexist,none,none,train
3,sexism2022_english-13021,woman?,not sexist,none,none,train
4,sexism2022_english-966,I bet she wished she had a gun,not sexist,none,none,dev


**Modify the dataset**

*Changed label_sexist from str to binary<br>
Split the data equaly, now we have equal number of sexist and nonsexist entries 9708 total*

In [ ]:
dummy = pd.get_dummies(df['label_sexist'])

# normalize the text and remove unnecessary columns
df2 = pd.concat((df, dummy), axis=1)
df2.drop(['label_sexist', 'not sexist', 'label_vector','rewire_id', 'label_category','split'], axis=1, inplace=True)
df2.rename(columns={'sexist':'label_sexist'}, inplace=True)
df['text'] = df['text'].astype(str)
df2['text'] = df2['text'].str.replace(r'\[USER\]|\[URL\]', '')

# this splits the data equaly 1:1
df2['label_sexist'].value_counts()
df_sexist = df2[df2['label_sexist'] == 1]
df_not_sexist = df2[df2['label_sexist'] == 0]
df_balanced = pd.concat([df_sexist, df_not_sexist.sample(df_sexist.shape[0])])
df_balanced['label_sexist'].value_counts()

df_balanced.head()

<ipython-input-2-564e488c0a8a>:8: FutureWarning: The default value of regex will change from True to False in a future version.
  df2['text'] = df2['text'].str.replace(r'\[USER\]|\[URL\]', '')


,text,label_sexist
6,Leg day is easy. Hot girls who wear miniskirt...,1
8,I get a new pussy every other week or whenever...,1
9,I agree with that but at the same time I know ...,1
14,O come on there's no way any men are attracted...,1
15,democrats from Minnesota (leftist muzzie shith...,1


**Encode the text and choose training dataset size**

Encode 1000 samples at a time and save them in a txt file (because of ram limitations)

In [ ]:
# change the size of the training dataset here (bigger number = more pain)
sample_df = df_balanced.sample(n=9708, random_state=42)

In [ ]:
# text_list = sample_df['text'].tolist()

In [ ]:
# preprocessed_text = bert_preprocess(sample_df['text'])
preprocessed_text = bert_preprocess(sample_df['text'])
encoded_text = bert_encoder(preprocessed_text)['pooled_output']

encoded_text = np.array(encoded_text)
# np.savetxt('encoded9000_9708.txt', encoded_text)
# print(encoded_text.shape)

Encode the text 1000 samples at a time then combine in to one file

In [ ]:
# import numpy as np

# # Load data from files
# data_file1 = np.loadtxt('/content/drive/MyDrive/datasets/encoded0_999.txt')

# data_file2 = np.loadtxt('/content/drive/MyDrive/datasets/encoded1000_1999.txt')

# data_file3 = np.loadtxt('/content/drive/MyDrive/datasets/encoded2000_2999.txt')

# data_file4 = np.loadtxt('/content/drive/MyDrive/datasets/encoded3000_3999.txt')

# data_file5 = np.loadtxt('/content/drive/MyDrive/datasets/encoded4000_4999.txt')

# data_file6 = np.loadtxt('/content/drive/MyDrive/datasets/encoded5000_5999.txt')

# data_file7 = np.loadtxt('/content/drive/MyDrive/datasets/encoded6000_6999.txt')

# data_file8 = np.loadtxt('/content/drive/MyDrive/datasets/encoded7000_7999.txt')

# data_file9 = np.loadtxt('/content/drive/MyDrive/datasets/encoded8000_8999.txt')

# data_file10 = np.loadtxt('/content/drive/MyDrive/datasets/encoded9000_9708.txt')

# # Concatenate the files
# encoded_file = np.concatenate((data_file1, data_file2, data_file3, data_file4, data_file5, data_file6, data_file7, data_file8, data_file9, data_file10), axis=0)

# # Save the concatenated data into a new file
# np.savetxt('encoded_text.txt', np.array(encoded_file))

# print(encoded_file.shape)

(9708, 768)


In [ ]:
# np.savetxt('label_sexist.txt', np.array(sample_df['label_sexist']))

**Neural network part**

In [ ]:
#split the data
from sklearn.model_selection import train_test_split

encoded_text = np.loadtxt('/content/drive/MyDrive/datasets/encoded_text.txt')
# label_sexist = np.loadtxt('/content/drive/MyDrive/datasets/label_sexist.txt')

X_train, X_test, y_train, y_test = train_test_split(encoded_text, sample_df['label_sexist'], stratify=sample_df['label_sexist'], test_size=0.2, random_state=123)

# Fix everything
X_train = np.array(X_train)
X_test = np.array(X_test)
y_train = np.array(y_train)
y_test = np.array(y_test)

**Model**

In [ ]:
model = ts.keras.Sequential([
    ts.keras.layers.Dense(256, activation='relu', input_shape=(768,)),
    ts.keras.layers.Dense(128, activation='relu'),
    ts.keras.layers.Dense(64, activation='relu'),
    ts.keras.layers.Dense(1, activation='sigmoid')
])


optimizer = ts.keras.optimizers.Adam()

model.compile(optimizer= optimizer,
              loss='binary_crossentropy',
              metrics=['accuracy'])

model.summary()

Model: "sequential_87"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_367 (Dense)           (None, 256)               196864    
                                                                 
 dense_368 (Dense)           (None, 128)               32896     
                                                                 
 dense_369 (Dense)           (None, 64)                8256      
                                                                 
 dense_370 (Dense)           (None, 1)                 65        
                                                                 
Total params: 238081 (930.00 KB)
Trainable params: 238081 (930.00 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
epochs = 8
batch_size = 30
model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, validation_data=(X_test, y_test))

loss, accuracy = model.evaluate(X_test, y_test)
print('Test Loss:', loss)
print('Test Accuracy:', accuracy)

Epoch 1/8
259/259 [==============================] - 4s 8ms/step - loss: 0.6406 - accuracy: 0.6279 - val_loss: 0.6031 - val_accuracy: 0.6859
Epoch 2/8
259/259 [==============================] - 2s 7ms/step - loss: 0.6010 - accuracy: 0.6749 - val_loss: 0.5957 - val_accuracy: 0.6797
Epoch 3/8
259/259 [==============================] - 2s 7ms/step - loss: 0.5843 - accuracy: 0.6907 - val_loss: 0.5917 - val_accuracy: 0.6952
Epoch 4/8
259/259 [==============================] - 2s 7ms/step - loss: 0.5814 - accuracy: 0.6899 - val_loss: 0.5792 - val_accuracy: 0.6998
Epoch 5/8
259/259 [==============================] - 2s 7ms/step - loss: 0.5683 - accuracy: 0.7038 - val_loss: 0.5855 - val_accuracy: 0.6962
Epoch 6/8
259/259 [==============================] - 3s 11ms/step - loss: 0.5672 - accuracy: 0.7054 - val_loss: 0.5837 - val_accuracy: 0.7024
Epoch 7/8
259/259 [==============================] - 3s 12ms/step - loss: 0.5527 - accuracy: 0.7123 - val_loss: 0.5859 - val_accuracy: 0.7008
Epoch 8/8
6